<a href="https://colab.research.google.com/github/kangsunwoo827/rl_23_project_team5/blob/main/Final_project_team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

from collections import deque
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import gym
from gym import spaces

In [ ]:
##########윷놀이 환경 설정#############
# 출발 전 말은 0, 나간 말은 30번째로.
#
#   11 10  9  8  7  6
#   12 26       21  5
#   13    27  22    4
#           23
#   14   24   28    3
#   15 25        29 2
#   16 17 18 19 20  1
#

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
empty_board='''
 -----------------------
| {}  {}  {}  {}  {}  {}|
|                       |
| {}   {}        {}   {}|
|        {}    {}       |
| {}                  {}|
|           {}          |
| {}                  {}|
|        {}    {}       |
| {}   {}        {}   {}|
|                       |
| {}  {}  {}  {}  {}  {}|
 -----------------------
 '''

board2num={
    11:0,10:1,9:2,8:3,7:4,6:5,
    12:6,26:7,       21:8,5:9,
          27:10,   22:11,
    13:12,               4:13,
              23:14,
    14:15,               3:16,
            24:17,28:18,
    15:19, 25:20,  29:21,2:22,
    16:23,17:24,18:25,19:26,20:27,1:28
 }

num2yut={
    1:'도',2:'개',3:'걸',4:'윷',5:'모',-1:'빽도'
}

In [ ]:
class YutEnv(gym.Env): # gym.Env를 상속
    def __init__(self):
      #뒤집어질 확률
      self.p=0.6
      self.p_lst=[
          3*(self.p**1)*((1-self.p)**3),
          6*(self.p**2)*((1-self.p)**2),
          4*(self.p**3)*((1-self.p)**1),
          1*(self.p**4)*((1-self.p)**0),
          1*(self.p**0)*((1-self.p)**4),
          1*(self.p**1)*((1-self.p)**3),
          ]
      #도 개 걸 윷 모 빽도

      
      self.pos= np.zeros(8,dtype=int)
      self._make_board()
      self.done=False
      self.team=1
      self.bonus=0
      self.turn_num=0

    def step(self, action):
        self.turn_num+=1
        print('{}번째 차례!'.format(self.turn_num))
        print('팀 {}의 차례입니다.'.format(self._num2team(self.team)))
        
        print('두구두구두구.... {}!!'.format(num2yut[self.move_num]))
        #random policy
        if not 0<=action<=3:
          raise Exception('action이 잘못됐습니다. cur_action is {}'.format(action))

        print('{}번째 말을 옮깁니다.'.format(action))
        prev_pos=self.pos[action]
        next_pos=self._move(self.pos[action],self.move_num)
        
        #윷, 모인 경우 한번 더!
        if self.move_num>=4:
          self.bonus=1
          
        for i in range(4):
          #업은 경우
          i=i*self.team
          if prev_pos>0 and self.pos[i]==prev_pos:
            self.pos[i]=next_pos

          #잡은경우
          if 0<next_pos<30 and self.pos[-(i+1)]==next_pos:
            print('잡았습니다!')
            self.pos[-(i+1)]=0
            self.bonus=1

        self.pos[action]=next_pos

        self._make_board()

        self._check_end()
        if self.done:
          print("team{} is win!".format(self._num2team(self.team)))


        reward = self.bonus #정의해야함.

        #팀 바꾸기
        if self.bonus:
          self.bonus=0
          print('한번 더!')
        else:
          self.pos=self.pos[::-1]
          self.team*=-1

        obs=self._get_state()
        

        return obs, reward, done

      
    def _check_end(self):
      if sum(self.pos[:4])==120 or sum(self.pos[-4:])==120:
        self.done=True

    def _move(self, prev_pos, move_num):
      #일단 빽도는 고려하지 않음.
      print('이동할양',move_num)
      for i in range(move_num):
        next_pos=self._move_one(prev_pos,short_cut=bool(i==0))
        prev_pos=next_pos
      return next_pos

    def _move_one(self,prev_pos,short_cut=False):
      if short_cut:
        if prev_pos==6:
          return 21
        elif prev_pos==11:
          return 26
        elif prev_pos==23:
          return 28
      if prev_pos in [1,20,30]:
        return 30
      elif prev_pos==0:
        return 2
      elif prev_pos==29:
        return 1
      else:
        return prev_pos+1

    def reset(self):
      self.pos= np.zeros(8,dtype=int)
      self._make_board()
      self.done=False
      self.team=1
      self.bonus=0
      self.turn_num=0
      print('reset')
      return self._get_state()


    def _make_board(self):
      # 앞에 4개는 우리팀 말들의 위치, 뒤 4개는 상대팀 말들의 위치
      self.board = np.zeros((8,31),dtype=int)
      print(self.pos)
      for i in range(8):
        self.board[i][self.pos[i]]=1
      return self.board
      
    def _throw_yut(self):
      #yut_num은 던졌을 때 뒤집어진 개수. 0이면 모, 4면 윷
      self.move_num=np.random.choice([1,2,3,4,5,-1], p=self.p_lst)

      #빽도 없애는 코드
      if self.move_num==-1:
        self.move_num=1
      
      return self.move_num

    def _get_state(self):
      #현재 보드 판 상태와 내가 움직여야 하는 개수 return
      return np.append(self.board.flatten(),self._throw_yut())

    def _num2team(self,team):
      return "★" if team>0 else '■'
    
    def render(self,not_draw=False):
      #내 말은 ● 상대 말은 X로 표시
      base=['◎']+['○']*4+['◎']+['○']*8+['◎']+['○']*8+['◎']+['○']*4+['◎']
      
      for i in range(4):
        if 0<self.pos[i]<30:
          base[board2num[self.pos[i]]]=self._num2team(self.team)
        if 0<self.pos[-(i+1)]<30:
          base[board2num[self.pos[-(i+1)]]]=self._num2team(-self.team)

      if not not_draw:
        print(empty_board.format(*base))
      return empty_board.format(*base)


      

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def random_policy(state):
  possible_action=[]
  for i in range(4):
    if not state[30*(i+1)]==1: #도착한 말이 아니라면
      possible_action.append(i)
  return np.random.choice(possible_action)

env=YutEnv()
env.render()
print('==')


[0 0 0 0 0 0 0 0]

 -----------------------
| ◎  ○  ○  ○  ○  ◎|
|                       |
| ○   ○        ○   ○|
|        ○    ○       |
| ○                  ○|
|           ◎          |
| ○                  ○|
|        ○    ○       |
| ○   ○        ○   ○|
|                       |
| ◎  ○  ○  ○  ○  ◎|
 -----------------------
 
==


In [ ]:
boards=[]
done=False
state=env.reset()
while not env.done:
  action=random_policy(state)
  state,return_value, done = env.step(action)
  boards.append(env.render(not_draw=False))

  if env.turn_num>100:
    break

[0 0 0 0 0 0 0 0]
reset
1번째 차례!
팀 ★의 차례입니다.
두구두구두구.... 개!!
2번째 말을 옮깁니다.
이동할양 2
[0 0 3 0 0 0 0 0]

 -----------------------
| ◎  ○  ○  ○  ○  ◎|
|                       |
| ○   ○        ○   ○|
|        ○    ○       |
| ○                  ○|
|           ◎          |
| ○                  ★|
|        ○    ○       |
| ○   ○        ○   ○|
|                       |
| ◎  ○  ○  ○  ○  ◎|
 -----------------------
 
2번째 차례!
팀 ■의 차례입니다.
두구두구두구.... 걸!!
0번째 말을 옮깁니다.
이동할양 3
[4 0 0 0 0 3 0 0]

 -----------------------
| ◎  ○  ○  ○  ○  ◎|
|                       |
| ○   ○        ○   ○|
|        ○    ○       |
| ○                  ■|
|           ◎          |
| ○                  ★|
|        ○    ○       |
| ○   ○        ○   ○|
|                       |
| ◎  ○  ○  ○  ○  ◎|
 -----------------------
 
3번째 차례!
팀 ★의 차례입니다.
두구두구두구.... 개!!
1번째 말을 옮깁니다.
이동할양 2
[0 3 3 0 0 0 0 4]

 -----------------------
| ◎  ○  ○  ○  ○  ◎|
|                       |
| ○   ○        ○   ○|
|        ○    ○       |
| ○                

In [ ]:
len(env._get_state())

In [ ]:
# def play(show_number):
#     env = YutEnv(show_number=show_number)
#     agents = [HumanAgent('O'),
#               HumanAgent('X')]
#     episode = 0
#     while True:
#         state = env.reset()
#         _, mark = state
#         done = False
#         env.render()
#         while not done:
#             agent = agent_by_mark(agents, next_mark(mark))
#             env.show_turn(True, mark)
#             ava_actions = env.available_actions()
#             action = agent.act(ava_actions)
#             if action is None:
#                 sys.exit()

#             state, reward, done, info = env.step(action)

#             print('')
#             env.render()
#             if done:
#                 env.show_result(True, mark, reward)
#                 break
#             else:
#                 _, mark = state
#         episode += 1